In [25]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [26]:
N_centr = 5
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,20,20,40,40,60,60,80,80,93]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="input/"
file_names=["my-m_ee_Run14AuAu_83rd_new_19842_1065runs.root"] 
#m_ee_Run14AuAu_83rd_new_19842_1065runs
#m_ee_Run14AuAu_81st_new_19831_785runs (hadrons) m_ee_Run14AuAu_82nd_new_19838_1026runs (hadrons reuse vtx)
#m_ee_Run14AuAu_79th_new_19821_1043runs m_ee_Run14AuAu_80th_new_19825_300runs (no vtx)
#m_ee_Run14AuAu_76th_new_19812_75runs m_ee_Run14AuAu_77th_new_19818_760runs #m_ee_Run14AuAu_78th_new_19820_500runs no vtx
#m_ee_Run14AuAu_74th_new_19784_984runs m_ee_Run14AuAu_75th_new_19794_741runs #hadrons
#m_ee_Run14AuAu_70th_new_19721_1065runs m_ee_Run14AuAu_73rd_new_19777_990runs
sim_file_path="../sim/output/Newembed/dca/"
dalitz_file_names=['piminus_50M_v5.root']
ccbar_name = ['piminus_50M.root']
sim_file_names = [dalitz_file_names[0], ccbar_name[0]]
legend_names = ["real","#pi^{-}","c#bar{c}"] 
hist_select_3D_names = ["DCA2_hist_0","DCA2_hist_1","DCA2_hist_2","DCA2_hist_3","DCA2_hist_4","sDCA2_hist_0","sDCA2_hist_1","sDCA2_hist_2","sDCA2_hist_3","sDCA2_hist_4",\
                        "DCA_2D_hist_0","DCA_2D_hist_1","DCA_2D_hist_2","DCA_2D_hist_3","DCA_2D_hist_4","sDCA_2D_hist_0","sDCA_2D_hist_1","sDCA_2D_hist_2","sDCA_2D_hist_3","sDCA_2D_hist_4"]

In [27]:
N_hists = len(hist_select_3D_names)
loc = True
Save_to_html = False

In [28]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(N_hists):
        hists_sim_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

input/my-m_ee_Run14AuAu_83rd_new_19842_1065runs.root
../sim/output/Newembed/dca/piminus_50M_v5.root
../sim/output/Newembed/dca/piminus_50M.root


In [29]:
conv_type = [1,25]
pt = [0.45,0.45]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"real_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dalitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    real_proj.Draw()
    piminus_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for dalitz_proj in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 179.34338347166
Standard deviation for dalitz_proj in centrality bin 0: 160.09385827803155
beam width = 80.83319699675646
Standard deviation for real_proj in centrality bin 1: 162.49684140556377
Standard deviation for dalitz_proj in centrality bin 1: 150.23444676034626
beam width = 61.92604034974003
Standard deviation for real_proj in centrality bin 2: 150.586212298528
Standard deviation for dalitz_proj in centrality bin 2: 146.12194508568044
beam width = 36.39484165090897
Standard deviation for real_proj in centrality bin 3: 151.798085880379
Standard deviation for dalitz_proj in centrality bin 3: 144.25001620681107
beam width = 47.271468152381985
Standard deviation for real_proj in centrality bin 4: 136.75344293205882
Standard deviation for dalitz_proj in centrality bin 4: 143.31460904012144
beam width = (2.6248419471128392e-15+42.866922102607155j)


In [30]:
conv_type = [1,25]
pt = [1.45,1.55]
c0 = root.TCanvas(f"c0",f"c0",1550,900)
c0.Divide(3,2)
projy = hists_read[0][0].ProjectionY(f"projy{0}")
projz = hists_read[0][0].ProjectionZ(f"projz{0}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    type_loc = icentr+5
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])
    ibinz0 = projz.FindBin(conv_type[0])-1
    ibinz1 = projz.FindBin(conv_type[1])-1
    real_proj = hists_read[0][type_loc].ProjectionX(hists_read[0][type_loc].GetName()+f"rsdeal_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="DCA_{T}, #mum",title_y="dN/dDCA,#mum^{-1}",offset_y=1.1,Lcolor=1, Lwidth=5)
    piminus_proj = hists_sim[0][type_loc].ProjectionX(hists_sim[1][type_loc].GetName()+f"dasdlitz_projproj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(piminus_proj,Lcolor=4, Msize=0)
    real_proj.Scale(1./(1+real_proj.Integral()))
    piminus_proj.Scale(1./(1+piminus_proj.Integral()))
    root.gPad.SetLogy()
    real_proj.GetXaxis().SetRangeUser(-1000,1000)
    real_proj.GetXaxis().SetNdivisions(505)
    real_proj.GetYaxis().SetRangeUser(0.000011,20)
    #real_proj.Fit("gaus", "", "", -200, 200)
    real_proj.Draw()
    piminus_proj.Draw("sameH")
    legends.append(root.TLegend(0.2,0.75,0.95,0.9))#f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"
    legends[-1].SetNColumns(3)
    legends[-1].AddEntry(real_proj,legend_names[0],"l")
    legends[-1].AddEntry(piminus_proj,legend_names[1],"l")
    Format_Legend(legends[-1])
    legends[-1].Draw()
    legends.append(root.TLegend(0.2,0.9,0.95,0.98,f"{central_bins[2*icentr]}-{central_bins[2*icentr+1]}%, pT={pt[0]}"))
    Format_Legend(legends[-1])
    legends[-1].Draw()
    # Calculate and print standard deviation
    print(f"Standard deviation for real_proj in centrality bin {icentr}: {real_proj.GetStdDev()}")
    print(f"Standard deviation for simul in centrality bin {icentr}: {piminus_proj.GetStdDev()}")
    print(f"beam width = {(real_proj.GetStdDev()**2-piminus_proj.GetStdDev()**2)**0.5}" )

c0.Draw()
c0.SaveAs(f"output/dca/sim.png")

Standard deviation for real_proj in centrality bin 0: 109.85365932310262
Standard deviation for simul in centrality bin 0: 101.50971562890624
beam width = 41.9952866357033
Standard deviation for real_proj in centrality bin 1: 106.01131416614614
Standard deviation for simul in centrality bin 1: 98.84596020010096
beam width = 38.312855327597184
Standard deviation for real_proj in centrality bin 2: 102.70188815383115
Standard deviation for simul in centrality bin 2: 96.78615503577846
beam width = 34.35284593381309
Standard deviation for real_proj in centrality bin 3: 101.4770557287312
Standard deviation for simul in centrality bin 3: 96.40599975701932
beam width = 31.677690102367162
Standard deviation for real_proj in centrality bin 4: 85.89427135619576
Standard deviation for simul in centrality bin 4: 95.81168045825885
beam width = (2.59934871960188e-15+42.450586102240216j)
